# Cases in Malaysia
> Updates on the respiratory illness that has infected more than one million people and killed tens of thousands.

- toc:false
- branch: master
- badges: false
- hide: false
- comments: false
- permalink:/covid-my-overview/

In [0]:
#hide
import pandas as pd
import numpy as np
import altair as alt
import ipywidgets as widgets
#from jinja2 import Template
from IPython.display import HTML

#hide
# COVID-19 Data from https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series
base_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data'

# confirmed, deaths, recovered
def get_time_series(type):
  _url = f'{base_url}/csse_covid_19_time_series/time_series_covid19_{type}_global.csv'
  _dff = pd.read_csv(_url)
  _cols = _dff.columns[~_dff.columns.isin(['Province/State', 'Country/Region', 'Lat', 'Long'])]
  _dff = (_dff.groupby('Country/Region')[_cols].sum().stack().reset_index(name = 'Cases')
        .rename(columns = {'level_1': 'Date', 'Country/Region': 'Country'}))
  _dff['Date'] = pd.to_datetime(_dff['Date'], format='%m/%d/%y')
  return _dff

# eg. 03-27-2020
def get_df_daily(date):
  _url = f'{base_url}/csse_covid_19_daily_reports/{date}.csv'
  return pd.read_csv(_url)

df_confirmed = get_time_series('confirmed')
df_recovered = get_time_series('recovered')
df_deaths = get_time_series('deaths')

date_latest = df_confirmed['Date'].max()

def countries_cases(df, n=5000, date_as_of=date_latest):
  df_latest = df[df['Date'].eq(date_as_of)]
  df_latest_n = df_latest[df_latest['Cases'] > n]
  return df_latest_n

# Last n days of type e.g. confirmed, recovered, deaths
df_last = lambda name, _df, n: _df[_df['Date'].isin(_df['Date'].tail(n))].set_index(['Country', 'Date']).rename(columns={'Cases': name})

# Last 2 days all global
df_summary = lambda last_n: pd.concat([df_last('Confirmed', df_confirmed, last_n), df_last('Deaths', df_deaths, last_n), df_last('Recovered', df_recovered, last_n)], axis=1)

df_global = df_summary(2).groupby(['Date']).sum().reset_index()

df_global_summary = df_global.stack().reset_index().rename(columns={'level_1': 'Type', 0: 'Cases'})

#print(date_latest)

s_date_latest = date_latest.strftime("%B %d, %Y") #.strftime('%m/%d/%Y')
n_confirmed = df_global.loc[1]['Confirmed']
n_deaths = df_global.loc[1]['Deaths']
n_recovered = df_global.loc[1]['Recovered']

df_global_diff = df_global.diff()
n_confirmed_diff = df_global_diff.loc[1]['Confirmed']
n_deaths_diff = df_global_diff.loc[1]['Deaths']
n_recovered_diff = df_global_diff.loc[1]['Recovered']

weekDays = ("Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday") 

df_my = lambda _df: _df[_df['Country'] == 'Malaysia'][['Date', 'Cases']]

df_confirmed_my = df_my(df_confirmed).rename(columns={'Cases': 'Cummulative Cases', 'Date': 'Reported Date'})
df_summary_my = df_summary(2).loc['Malaysia'].reset_index(drop=True)
n_confirmed_my = df_summary_my.loc[1]['Confirmed']
n_deaths_my = df_summary_my.loc[1]['Deaths']
n_recovered_my = df_summary_my.loc[1]['Recovered']

df_summary_my_diff = df_summary_my.diff()
n_confirmed_my_diff = df_summary_my_diff.loc[1]['Confirmed']
n_deaths_my_diff = df_summary_my_diff.loc[1]['Deaths']
n_recovered_my_diff = df_summary_my_diff.loc[1]['Recovered']


def make_pkp_line(date, label, max):
  _df = pd.DataFrame({'Date': [date, date], 'Cases': [0, max]})
  _base_ref = alt.Chart(_df).encode(x='Date:T', y='Cases:Q')
  return (_base_ref.mark_line(color='black', opacity=.5, strokeDash=[3,3])  + 
          _base_ref.transform_filter(alt.datum['Cases'] > 0).mark_text(text=label, dx=-20, dy=-10, angle=270))

_pkp_line = lambda n: make_pkp_line('2020-03-18', 'PKP1', n) + make_pkp_line('2020-04-01', 'PKP2', n) + make_pkp_line('2020-04-15', 'PKP3', n)



## COVID-19: Malaysia at a Glance

In [2]:
#hide_input
HTML(
    f'<div style="height: 30px; width: 600px; margin: 0 auto;"><span style="font-size:0.8em;">as of {s_date_latest} 12PM</span></div>'
    f'<div style="width: 600px; margin: 0 auto;">'
    f'<div style="float: left; width: 200px">Confirmed Cases<br/><h1>{n_confirmed_my:,}</h1>(+{n_confirmed_my_diff:,.0f})</div>'
    f'<div style="float: right; width: 200px">Deaths<br/><h1>{n_deaths_my:,}</h1>{n_deaths_my/n_confirmed_my:.2%} (+{n_deaths_my_diff:,.0f})</div>'
    f'<div style="float: right; width: 200px">Recovered<br/><h1>{n_recovered_my:,}</h1>{n_recovered_my/n_confirmed_my:.2%} (+{n_recovered_my_diff:,.0f})</div>'
    f'</div>'
)

In [29]:
#hide_input
_confirmed_bar = alt.Chart(df_confirmed_my).mark_bar().encode(
    x='Reported Date:T',
    y='Cummulative Cases:Q',
    tooltip=list(df_confirmed_my)
).properties(
    width=600,
    title=f'Cummulative COVID-19 cases in Malaysia (n = {n_confirmed_my})'
)

(_confirmed_bar + _pkp_line(n_confirmed_my + 1))

alt.LayerChart(...)

### Is the curve is flatterning in Malaysia?
> Inflection-sensitive chart for detecting successful interventions, from the article "How To Tell If We're Beating COVID-19".

In [23]:
#hide_input
df_confirmed_my['New Cases'] = df_confirmed_my['Cummulative Cases'].diff()
_data = df_confirmed_my[(df_confirmed_my['Cummulative Cases'] > 100) & df_confirmed_my['New Cases'] > 0]
_curve_line = alt.Chart(_data).mark_line().encode(
    alt.X('Cummulative Cases:Q', scale=alt.Scale(type='log'), title = 'Cummulative Confirmed Cases (Log Scale)'), 
    alt.Y('New Cases:Q', scale=alt.Scale(type='log'), title = 'Daily New Confirmed Cases (Log Scale)')
).properties(width = 600)

df_confirmed_my[df_confirmed_my['Reported Date'].isin(['2020-03-18', '2020-04-01', '2020-04-15'])]
_curve_line

alt.Chart(...)

Please refer _minutephysics_ for [How To Tell If We're Beating COVID-19](https://youtu.be/54XLXg4fYsc)

In [5]:
#hide
df_confirmed_my[['Reported Date', 'Cummulative Cases']].set_index('Reported Date').rename(columns={'Cummulative Cases': 'Total'}).transpose()

Reported Date,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,2020-01-31,2020-02-01,2020-02-02,2020-02-03,2020-02-04,2020-02-05,2020-02-06,2020-02-07,2020-02-08,2020-02-09,2020-02-10,2020-02-11,2020-02-12,2020-02-13,2020-02-14,2020-02-15,2020-02-16,2020-02-17,2020-02-18,2020-02-19,2020-02-20,2020-02-21,2020-02-22,2020-02-23,2020-02-24,2020-02-25,2020-02-26,2020-02-27,2020-02-28,2020-02-29,2020-03-01,...,2020-03-14,2020-03-15,2020-03-16,2020-03-17,2020-03-18,2020-03-19,2020-03-20,2020-03-21,2020-03-22,2020-03-23,2020-03-24,2020-03-25,2020-03-26,2020-03-27,2020-03-28,2020-03-29,2020-03-30,2020-03-31,2020-04-01,2020-04-02,2020-04-03,2020-04-04,2020-04-05,2020-04-06,2020-04-07,2020-04-08,2020-04-09,2020-04-10,2020-04-11,2020-04-12,2020-04-13,2020-04-14,2020-04-15,2020-04-16,2020-04-17,2020-04-18,2020-04-19,2020-04-20,2020-04-21,2020-04-22
Total,0,0,0,3,4,4,4,7,8,8,8,8,8,10,12,12,12,16,16,18,18,18,19,19,22,22,22,22,22,22,22,22,22,22,22,22,23,23,25,29,...,238,428,566,673,790,900,1030,1183,1306,1518,1624,1796,2031,2161,2320,2470,2626,2766,2908,3116,3333,3483,3662,3793,3963,4119,4228,4346,4530,4683,4817,4987,5072,5182,5251,5305,5389,5425,5482,5532


In [27]:
#hide_input
rolling_window = 7
df_reported_my = df_confirmed_my[['Reported Date', 'Cummulative Cases']].set_index('Reported Date').diff().rename(columns={'Cummulative Cases': 'Reported Cases'}).reset_index()
df_reported_my[f'Moving Average'] = df_reported_my.iloc[:, 1].rolling(window=rolling_window).mean()

_new_bar = alt.Chart(df_reported_my).mark_bar().encode(
    x='Reported Date:T',
    y='Reported Cases:Q',
    tooltip = list(df_reported_my)
)

_average_line = alt.Chart(df_reported_my).mark_line(color = 'red').encode(
    x='Reported Date:T',
    y='Moving Average:Q',
    tooltip = list(df_reported_my)
)

(_new_bar + _average_line + _pkp_line(df_reported_my['Reported Cases'].max() + 1)).properties(
    title = f'Daily Reported Cases (with {rolling_window} days average) in Malaysia',
    width = 600
)

TypeError: ignored

In [7]:
#hide
df_reported_my.transpose()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91
Reported Date,2020-01-22,2020-01-23 00:00:00,2020-01-24 00:00:00,2020-01-25 00:00:00,2020-01-26 00:00:00,2020-01-27 00:00:00,2020-01-28 00:00:00,2020-01-29 00:00:00,2020-01-30 00:00:00,2020-01-31 00:00:00,2020-02-01 00:00:00,2020-02-02 00:00:00,2020-02-03 00:00:00,2020-02-04 00:00:00,2020-02-05 00:00:00,2020-02-06 00:00:00,2020-02-07 00:00:00,2020-02-08 00:00:00,2020-02-09 00:00:00,2020-02-10 00:00:00,2020-02-11 00:00:00,2020-02-12 00:00:00,2020-02-13 00:00:00,2020-02-14 00:00:00,2020-02-15 00:00:00,2020-02-16 00:00:00,2020-02-17 00:00:00,2020-02-18 00:00:00,2020-02-19 00:00:00,2020-02-20 00:00:00,2020-02-21 00:00:00,2020-02-22 00:00:00,2020-02-23 00:00:00,2020-02-24 00:00:00,2020-02-25 00:00:00,2020-02-26 00:00:00,2020-02-27 00:00:00,2020-02-28 00:00:00,2020-02-29 00:00:00,2020-03-01 00:00:00,...,2020-03-14 00:00:00,2020-03-15 00:00:00,2020-03-16 00:00:00,2020-03-17 00:00:00,2020-03-18 00:00:00,2020-03-19 00:00:00,2020-03-20 00:00:00,2020-03-21 00:00:00,2020-03-22 00:00:00,2020-03-23 00:00:00,2020-03-24 00:00:00,2020-03-25 00:00:00,2020-03-26 00:00:00,2020-03-27 00:00:00,2020-03-28 00:00:00,2020-03-29 00:00:00,2020-03-30 00:00:00,2020-03-31 00:00:00,2020-04-01 00:00:00,2020-04-02 00:00:00,2020-04-03 00:00:00,2020-04-04 00:00:00,2020-04-05 00:00:00,2020-04-06 00:00:00,2020-04-07 00:00:00,2020-04-08 00:00:00,2020-04-09 00:00:00,2020-04-10 00:00:00,2020-04-11 00:00:00,2020-04-12 00:00:00,2020-04-13 00:00:00,2020-04-14 00:00:00,2020-04-15 00:00:00,2020-04-16 00:00:00,2020-04-17 00:00:00,2020-04-18 00:00:00,2020-04-19 00:00:00,2020-04-20 00:00:00,2020-04-21 00:00:00,2020-04-22 00:00:00
Reported Cases,NaT,0,0,3,1,0,0,3,1,0,0,0,0,2,2,0,0,4,0,2,0,0,1,0,3,0,0,0,0,0,0,0,0,0,0,0,1,0,2,4,...,41,190,138,107,117,110,130,153,123,212,106,172,235,130,159,150,156,140,142,208,217,150,179,131,170,156,109,118,184,153,134,170,85,110,69,54,84,36,57,50
Moving Average,NaT,NaN,NaN,NaN,NaN,NaN,NaN,1,1.14286,1.14286,0.714286,0.571429,0.571429,0.857143,0.714286,0.571429,0.571429,1.14286,1.14286,1.42857,1.14286,0.857143,1,1,0.857143,0.857143,0.571429,0.571429,0.571429,0.428571,0.428571,0,0,0,0,0,0.142857,0.142857,0.428571,1,...,20.7143,47,64.1429,77.7143,91.5714,107.286,119,135,125.429,136,135.857,143.714,161.571,161.571,162.429,166.286,158.286,163.143,158.857,155,167.429,166.143,170.286,166.714,171,173,158.857,144.714,149.571,145.857,146.286,146.286,136.143,136.286,129.286,110.714,100.857,86.8571,70.7143,65.7143


In [8]:
#hide_input
df_summary_my = pd.merge(df_my(df_recovered), df_my(df_deaths), on='Date').rename(columns={'Cases_x': 'Recovered', 'Cases_y': 'Deaths'})
df_summary_my['Close'] = df_summary_my['Recovered'] + df_summary_my['Deaths']
df_summary_my['Recovered_Rate'] = df_summary_my['Recovered']/ df_summary_my['Close']
df_summary_my['Deaths_Rate'] = df_summary_my['Deaths']/ df_summary_my['Close']
df_summary_my = pd.merge(df_summary_my, df_confirmed_my.rename(columns={'Reported Date': 'Date' ,'Cummulative Cases': 'Reported'}), on = 'Date')
df_summary_my['Active'] = df_summary_my['Reported'] - df_summary_my['Close']
df_summary_my['Active Mean'] = df_summary_my['Active'].rolling(window=rolling_window).mean()

_active_bar = alt.Chart(df_summary_my).mark_bar().encode(
    x='Date:T',
    y='Active:Q',
    tooltip=list(df_summary_my)
)

_mean_line = alt.Chart(df_summary_my).mark_line(color='red').encode(
    x='Date:T',
    y='Active Mean:Q',
    tooltip=list(df_summary_my)
)

(_active_bar + _mean_line + _pkp_line(df_summary_my['Active'].max() + 1)).properties(
    title = f'Daily Active Cases (with {rolling_window} days average) in Malaysia',
    width=600
)


alt.LayerChart(...)

In [9]:
#hide_input
_base = alt.Chart(df_summary_my[['Date', 'Recovered_Rate', 'Deaths_Rate']]).encode(x='Date:T')
_recovered = _base.mark_line(color='green').encode(
              y=alt.Y('Recovered_Rate:Q', axis=alt.Axis(format='%')))
_deaths = _base.mark_line(color='red').encode(y=alt.Y('Deaths_Rate:Q', axis=alt.Axis(format='%')))

(_recovered + _deaths + _pkp_line(1.0)).properties(title='Outcome of Total Closed Case (Recovery Rate vs Death Rate)', width=600)

alt.LayerChart(...)

In [10]:
#hide
df_states = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vTzT9vUJNiKV2yN4sb_VvxKcq-B2triWGPE74rfUT4XOsF-5qsB1tM6OfMPVKiRHX95tE9tPubdTbxY/pub?gid=1726267961&single=true&output=csv", parse_dates=['Date'])
df_states.set_index('Date').sum(axis=1)

Date
3/27    130
3/28    159
3/29    150
3/30    156
3/31    140
4/1     142
4/2     208
4/3     217
4/4     150
4/5     179
4/6     131
4/7     170
4/8     156
4/9     109
4/10    118
4/11    184
4/12    153
4/13    134
4/14    170
4/15     85
4/16    110
4/17     69
4/18     54
4/19     84
4/20     36
4/21     57
4/22     50
dtype: int64

In [11]:
#hide
df_districts = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vTzT9vUJNiKV2yN4sb_VvxKcq-B2triWGPE74rfUT4XOsF-5qsB1tM6OfMPVKiRHX95tE9tPubdTbxY/pub?gid=1667946793&single=true&output=csv")
col_latest = df_districts.columns.values[-1]

df_districts[['Districts', 'State', col_latest]].sort_values(col_latest, ascending=False).reset_index(drop=True).head(10)

,Districts,State,4/20
0,Lembah Pantai,KUL,629
1,Hulu Langat,SGR,446
2,Petaling,SGR,374
3,Seremban,NSN,307
4,Kuching,SRW,264
5,Kluang,JHR,222
6,Johor Bahru,JHR,193
7,Klang,SGR,173
8,Kepong,KUL,159
9,Gombak,SGR,144


In [12]:
#hide
df_districts_last2 = df_districts.set_index(['Districts', 'State']).transpose().tail(2)
df_districts_new = df_districts_last2.diff().tail(1).transpose()
df_districts_new = df_districts_new.sort_values(by=df_districts_new.columns[0], ascending=False).head(10)
df_districts_new[df_districts_new[df_districts_new.columns[0]] > 0]

,,4/20
Districts,State,
Putrajaya,PJY,13.0
Kuching,SRW,4.0
Seremban,NSN,3.0
Kuala Langat,SGR,2.0
Bintulu,SRW,2.0
Kepong,KUL,2.0
Kuantan,PHG,2.0
Johor Bahru,JHR,1.0
Melaka Tengah,MLK,1.0
